In [8]:
import pickle
import os
import sys
import json
from moralstrength.moralstrength import string_moral_values
from project.utils.classes.tokenizer import Tokenizer
import warnings
warnings.filterwarnings("ignore")

#### Read in the Reddit Data

In [9]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))

pickle_file_path_comment = os.path.join(parent_directory,
    'data_collection/project_data/climateCommentsDf.pickle')

pickle_file_path_submission = os.path.join(parent_directory,
    'data_collection/project_data/climateSubmissionsDf.pickle')

with open(pickle_file_path_comment, 'rb') as f:
    comments_df = pickle.load(f)

with open(pickle_file_path_submission, 'rb') as f:
    submission_df = pickle.load(f)

/Users/kathryn/Projects/Advanced-ML/project/climate-conversations/project
/Users/kathryn/Projects/Advanced-ML/project/climate-conversations/project/data_collection/project_data/climateCommentsDf.pickle


In [ ]:
#comments_df.head()
subset_comments_df = comments_df.iloc[0:1000]
subset_comments_df.head()

,id,subreddit,body,author,score,gilded,created_utc,parent_id,link_id,retrieved_on,controversiality,is_submitter
0,inlvurb,politics,[removed],[deleted],1,0,1662654718,x8y56x,x8y56x,1665086767,0,False
1,inlw0wi,worldnews,Needed a laugh!,serious-activity-228,3,0,1662654781,inlvhve,x8yhv5,1665086761,0,False
2,inlx8g7,politics,[deleted],[deleted],1,0,1662655233,inkxrse,x8y56x,1665086718,0,False
3,inlxwat,worldnews,Asked!,dieyoufool3,3,0,1662655481,inlszmb,x94ag0,1665086696,0,True
4,inlyusw,worldnews,[deleted],[deleted],1,0,1662655839,inlyqzt,x8yhv5,1665086662,0,False


In [ ]:
#comments_df.to_csv("comments.csv")
subset_comments_df.to_csv("comments.csv")

In [ ]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(parent_directory)

tokenizer = Tokenizer(filepath='comments.csv',
                      filename='tokenized_comments.csv')
tokenizer.process(cols_to_tokenize=[('body', 'tokenized_body')])

#### Method 1: Assign Moral Foundations based on Word Counts

In [ ]:
def assign_moral_foundations(comment, extended_dict):
    foundations = {}
    for word in comment:
        for key, foundation_values in extended_dict.items():
            # Ensure foundation_values is a list for consistency
            if not isinstance(foundation_values, list):
                foundation_values = [foundation_values]
            for value in foundation_values:
                if key.endswith('*') and word.startswith(key[:-1]):
                    # Increment the foundation's count
                    foundations[value] = foundations.get(value, 0) + 1
                elif word == key:
                    # Increment the foundation's count
                    foundations[value] = foundations.get(value, 0) + 1
    return foundations

with open('expanded_moral_foundations_dictionary.json', 'r') as f:
    word_to_moral_foundation_expanded = json.load(f)
# Assuming tokenizer.tokenized_df['tokenized_body_words_norm']
# contains lists of words for each comment
tokenizer.tokenized_df['moral_foundations'] = \
tokenizer.tokenized_df['tokenized_body_words_norm'].apply(
    lambda x: assign_moral_foundations(x, word_to_moral_foundation_expanded)
)

#### Method 2: Using moralstrength package

In [ ]:
texts = tokenizer.tokenized_df['tokenized_body_sents']

for text in texts:
    text = text
    print(text)
    #result = estimate_morals(text, process=True)
    result = string_moral_values(text[0], model='unigram+freq')
    print(result)